# Ensemble of Object Detection Models based on Detectron2

## Prepare Development Environment
1. Environments
    * GPU: RTX3070 8GB
    * OS: Ubuntu20.04
    * CUDA: 11.3
    * Pytorch==1.10
    * Detectron2==0.6

2. Installation
    ```bash
    sudo apt-get install -y python3-dev python3-venv
    python3 -m venv env
    source env/bin/activate
    python -m pip install pip -U
    python -m pip install -r requirements.txt
    python -m ipykernel install --user --name env --display-name ensemble_detectron2
    python -m pip install "git+https://github.com/facebookresearch/detectron2@v0.6"
    ```

## Download Dataset

In [7]:
!wget -c http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!rm val2017.zip
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm annotations_trainval2017.zip

## Import Modules

In [1]:
import torch
from tqdm import tqdm

from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.evaluation.coco_evaluation import COCOEvaluator
from detectron2.data.build import build_detection_test_loader
from detectron2.evaluation.evaluator import inference_on_dataset
from detectron2.evaluation.evaluator import inference_context
from detectron2.structures import Instances, Boxes

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import ensemble_boxes

## Prepare Models, Data and Evaluation

In [2]:
register_coco_instances("dataset_val", {}, "./annotations/instances_val2017.json", "./val2017")

In [3]:
model_configs = [
    "faster_rcnn_R_50_C4_1x.yaml",
    "faster_rcnn_R_50_DC5_1x.yaml",
    "retinanet_R_50_FPN_1x.yaml",
]

models = dict()
for config in model_configs:
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(f"COCO-Detection/{config}"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(f"COCO-Detection/{config}")
    cfg.DATASETS.VAL = ("dataset_val",)

    models[config] = DefaultPredictor(cfg).model

cfg = get_cfg()
cfg.DATASETS.VAL = ("dataset_val",)
val_loader = build_detection_test_loader(cfg, "dataset_val")

Loading config /home/kyungpyo/git/Ensemble-Object-Detection-using-Detectron2/env/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



## Evaluate Baseline Models

In [4]:
for config, model in models.items():
    evaluator = COCOEvaluator("dataset_val", False, output_dir=f"results/{config.split('.')[0]}")
    evaluator.reset()
    with inference_context(model), torch.no_grad():
        iter = tqdm(val_loader, total=len(val_loader))
        for idx, inputs in enumerate(iter):
            outputs = model(inputs)
            torch.cuda.synchronize()
            evaluator.process(inputs, outputs)

    print("\n================================================================\n")
    print(config)
    print("\n================================================================\n")
    results = evaluator.evaluate()
    print("\n================================================================\n")

  0%|          | 0/5000 [00:00<?, ?it/s]/home/kyungpyo/git/Ensemble-Object-Detection-using-Detectron2/env/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 5000/5000 [10:35<00:00,  7.87it/s]




faster_rcnn_R_50_C4_1x.yaml


Loading and preparing results...
DONE (t=0.92s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.357
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.311
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.563
 Averag

100%|██████████| 5000/5000 [06:10<00:00, 13.50it/s]




faster_rcnn_R_50_DC5_1x.yaml


Loading and preparing results...
DONE (t=0.68s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.373
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.587
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.417
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.488
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.511
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.299
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.565
 Avera

  0%|          | 0/5000 [00:00<?, ?it/s]/home/kyungpyo/git/Ensemble-Object-Detection-using-Detectron2/env/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
100%|██████████| 5000/5000 [05:09<00:00, 16.17it/s]




retinanet_R_50_FPN_1x.yaml


Loading and preparing results...
DONE (t=0.99s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.567
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.231
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.483
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.319
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.518
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.551
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.372
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.592
 Average

## Box mAP of Baseline Models

| Baselines                   | Box AP @(IoU=0.50:0.95, area=all, maxDets=100) |
|-----------------------------|------|
|faster_rcnn_R_50_C4_1x.yaml  | 0.357|
|faster_rcnn_R_50_DC5_1x.yaml | 0.373|
|retinanet_R_50_FPN_1x.yaml   | 0.374|

## Ensemble using Weighted Boxes Fusion Method

### Load Detection Results and Ground Truth

In [4]:
gt_path = "./annotations/instances_val2017.json"
coco_gt = COCO(gt_path)

dt_paths = [
    "./results/faster_rcnn_R_50_C4_1x/coco_instances_results.json",
    "./results/faster_rcnn_R_50_DC5_1x/coco_instances_results.json",
    "./results/retinanet_R_50_FPN_1x/coco_instances_results.json",
]

coco_dts = [coco_gt.loadRes(dt_path) for dt_path in dt_paths]
img_ids = coco_gt.getImgIds()

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.62s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.33s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.95s)
creating index...
index created!


In [6]:
# parameters
iou_thr = 0.7
skip_box_thr = 0.0001

print("\n================================================================\n")
print("Ensemble using Weighted Boxes Fusion")
print(f"iou thr: {iou_thr}, skip_box_thr: {skip_box_thr}")
print("\n================================================================\n")

ensemble = []
cnt_id = 0
for img_id in img_ids:
    height = float(coco_gt.loadImgs(img_id)[0]["height"])
    width = float(coco_gt.loadImgs(img_id)[0]["width"])

    tmp_anns = []
    boxes_list = []
    scores_list = []
    labels_list = []

    for coco_dt in coco_dts:
        boxes = []
        scores = []
        labels = []
        for ann in coco_dt.imgToAnns[img_id]:
            x1, y1 = ann["bbox"][0], ann["bbox"][1]
            x2 = ann["bbox"][0] + ann["bbox"][2]
            y2 = ann["bbox"][1] + ann["bbox"][3]
            x1, x2 = x1/width, x2/width
            y1, y2 = y1/height, y2/height

            x1 = min(1.000, max(0.000, x1))
            x2 = min(1.000, max(0.000, x2))
            y1 = min(1.000, max(0.000, y1))
            y2 = min(1.000, max(0.000, y2))
                
            boxes.append([x1,y1,x2,y2])
            scores.append(ann["score"])
            labels.append(ann["category_id"])

        boxes_list.append(boxes)
        scores_list.append(scores)
        labels_list.append(labels)
    
    boxes, scores, labels = ensemble_boxes.weighted_boxes_fusion(
                                            boxes_list, 
                                            scores_list, 
                                            labels_list, 
                                            weights=None, 
                                            iou_thr=iou_thr, 
                                            skip_box_thr=skip_box_thr)

    for box, score, label in zip(boxes, scores, labels):
        x1, y1, x2, y2 = box
        
        x1 *= width
        x2 *= width
        y1 *= height
        y2 *= height

        ann = dict(
            image_id=img_id,
            category_id=label,
            bbox=[x1, y1, x2-x1, y2-y1],
            score=score,
            id=cnt_id,
        )

        ensemble.append(ann)
        
        cnt_id += 1

coco_ensemble = coco_gt.loadRes(ensemble)
coco_eval = COCOeval(coco_gt, coco_ensemble, "bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
print("\n================================================================\n")



Ensemble using Weighted Boxes Fusion
iou thr: 0.7, skip_box_thr: 0.0001


Loading and preparing results...
DONE (t=1.45s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=40.65s).
Accumulating evaluation results...
DONE (t=8.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.601
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.453
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.334
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.545
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]

## Results

| Models                   | Box AP @(IoU=0.50:0.95, area=all, maxDets=100) |
|-----------------------------|------|
|faster_rcnn_R_50_C4_1x.yaml  | 0.357|
|faster_rcnn_R_50_DC5_1x.yaml | 0.373|
|retinanet_R_50_FPN_1x.yaml   | 0.374|
|__Ensemble Model__   | __*0.403 (+0.029)*__|